In [1]:
import requests
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch.nn.functional as F
import torch
import numpy as np

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "gpt2-large"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token =  tokenizer.eos_token

In [3]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3840, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=1280)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=5120, nx=1280)
          (c_proj): Conv1D(nf=1280, nx=5120)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = model.to(device)

# arrivée des données

In [4]:
url = 'https://rome.baulab.info/data/dsets/known_1000.json'
response = requests.get(url) 
data = response.json()

In [5]:
nb_prompt = 100
prompts = [dict['prompt'] for dict in data][:nb_prompt]
subjects = [dict['subject'] for dict in data][:nb_prompt]
input= tokenizer(prompts, return_tensors="pt", padding= True, return_offsets_mapping= True)#return_offsets_mapping pour retenir ou sont les tokens
input = input.to(device)#input est un dico composé de deux clefs : input_ids et attention_mask, inputs_ids a pour valeur un tenseur (nb_prompt x nb_max token) donnant la position 
#de chaque token et attention mask est aussi un tenseur (nb_prompt x nb_max token) donnant 1 si c'est PAS du padding, et 0 sinon 


In [6]:
print(prompts[0])
data[0]["attribute"]
input['attention_mask'][0]

Vinson Massif is located in the continent of


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

In [7]:
mask = []
for j, prompt in enumerate(prompts):
    map = torch.zeros_like(input.input_ids[j], dtype=torch.int)#input_ids = id du token, fait un tenseur de 0 de la même dimension
    for i,t in enumerate(input.offset_mapping[j]):#offset_mapping = où est-ce qu'on a mis le padding, i = position, 
        
        if (prompts[j].find(subjects[j])-1<=t[0]) and (t[1]<=prompts[j].find(subjects[j])+len(subjects[j])):#sélectionne aussi le padding, qu'on élimine avec logical
            map[i] = 1
    mask.append(map)
masks_tensor = torch.stack(mask)
masks_tensor = torch.logical_and(masks_tensor, input.attention_mask).int()

In [8]:
input.offset_mapping.shape #n prompt,le plus gros prompt fait m tokens, (position de début du token, position de fin du token)

torch.Size([100, 21, 2])

In [9]:
def noise_hook(module,input,output):
    noise = torch.randn_like(output)*5
    noisy_output = output + noise * masks_tensor.unsqueeze(-1).float()
    print(output.shape, masks_tensor.shape)

    return noisy_output

In [10]:
# fonction qui sert à obtenir le logits du dernier non-padding token
def last_non_padding_token_logits(logits, attention_mask):
    """récupère un tenseur logits, et attention mask, et retourne un tenseur donnant, pour chauque logit de chaque prompt, le logit du dernier mots
    """
    # For each input, find the last non-padding token
    last_non_padding_logits = []
    
    for i in range(logits.size(0)):  # Loop over each prompt in the batch
        # Find the last non-padding token position
        non_padding_positions = (attention_mask[i] == 1).nonzero(as_tuple=True)[0]
        last_non_padding_token_index = non_padding_positions[-1]
        
        # Get the logits of the last non-padding token
        last_non_padding_logits.append(logits[i, last_non_padding_token_index])
    last_non_padding_logits = torch.stack(last_non_padding_logits)
    return last_non_padding_logits

In [11]:
#sans le noise
input= tokenizer(prompts, return_tensors="pt", padding= True).to(device)
with torch.no_grad():
    outputs = model(**input, output_hidden_states = False, output_attentions =True)

#outputs["logits"] est un tenseur de (nb_prompt = batch size, nb_max token, proba brut) de taille (batch size,sequence_length, vocab_size), qui donne pour chaque token, 
# de chaque prompt la probabilité du mot suivant, ie prompt = "le chat mange la souris" le logits du token "chat" peut être égale à "mange" : 4.5 "dort" : 2.3,"joue" : 3.1
# "souris" : 1.2, "chien" : 0.5


`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [12]:
outputs["logits"].shape

torch.Size([100, 21, 50257])

In [13]:
#on enregistre le logit no_noise 

logits_no_noise = last_non_padding_token_logits(outputs.logits,input.attention_mask)

logits_no_noise.shape


torch.Size([100, 50257])

# Avec bruit

In [14]:
masked_noise_hook = model.transformer.drop.register_forward_hook(noise_hook)

input= tokenizer(prompts, return_tensors="pt", padding= True).to(device)
with torch.no_grad():
    outputs = model(**input, labels = input.input_ids, output_hidden_states = False, output_attentions =False)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


torch.Size([100, 21, 1280]) torch.Size([100, 21])


In [15]:
logits_noise = last_non_padding_token_logits(outputs.logits,input.attention_mask)


In [16]:
masked_noise_hook.remove()

In [19]:
def diff_proba(data, logits_no_noise, logits_noise, tokenizer, nb_prompt):
    """
    Calcule la différence de probabilité de l'attribut (le mot à prédire) entre les conditions avec et sans bruit.

    :param data: Liste de dictionnaires contenant les prompts et les attributs à prédire.
    :param logits_no_noise: Logits des prompts sans bruit.
    :param logits_noise: Logits des prompts avec bruit.
    :param tokenizer: Tokenizer utilisé pour encoder les mots.
    :return: Liste de dictionnaires mise à jour avec la différence de probabilité.
    """
    # Calcul des probabilités pour chaque condition
    probs_no_noise = F.softmax(logits_no_noise, dim=-1)
    probs_noise = F.softmax(logits_noise, dim=-1)

    for index, prompt_dict in enumerate(data[:nb_prompt]):
        predicted_word = prompt_dict["attribute"]
        predicted_ids = tokenizer.encode(predicted_word, add_special_tokens=False)

        # On suppose que le mot à prédire est un seul token
        token_id = predicted_ids[0]

        # Obtenir les probabilités pour le token prédit
        p_no_noise = probs_no_noise[index, token_id].item()
        p_noise = probs_noise[index, token_id].item()

        # Calculer la différence de probabilité
        prompt_dict["attr_prob_diff"] = p_noise - p_no_noise

    return data


In [20]:
diff_proba(data, logits_no_noise, logits_noise,tokenizer, nb_prompt)

[{'known_id': 0,
  'subject': 'Vinson Massif',
  'attribute': 'Antarctica',
  'template': '{} is located in the continent',
  'prediction': ' of Antarctica. It is the largest of the three',
  'prompt': 'Vinson Massif is located in the continent of',
  'relation_id': 'P30',
  'attr_prob_diff': -1.425725059789329e-06},
 {'known_id': 1,
  'subject': 'Beats Music',
  'attribute': 'Apple',
  'template': '{} is owned by',
  'prediction': ' Apple, which is also the owner of Beats Electronics',
  'prompt': 'Beats Music is owned by',
  'relation_id': 'P127',
  'attr_prob_diff': -2.0328182046114307e-05},
 {'known_id': 2,
  'subject': 'Audible.com',
  'attribute': 'Amazon',
  'template': '{} is owned by',
  'prediction': ' Amazon.com, Inc. or its affiliates.',
  'prompt': 'Audible.com is owned by',
  'relation_id': 'P127',
  'attr_prob_diff': -1.5734315866211546e-05},
 {'known_id': 3,
  'subject': 'The Big Bang Theory',
  'attribute': 'CBS',
  'template': '{} premieres on',
  'prediction': ' CBS 